In [ ]:
import tweepy #,csv,re,sys
from textblob import TextBlob
import matplotlib.pyplot as plt


class SentimentAnalysis:

    def __init__(self):
        self.tweets = []
        self.tweetText = []

    def DownloadData(self):
        # authenticating
        consumerKey = 'HERE YOUR KEY'
        consumerSecret = 'HERE YOUR SECRET KEY'
        accessToken = 'HERE YOUR TOKEN'
        accessTokenSecret = 'HERE YOUR SECRET TOKEN'
        auth = tweepy.OAuthHandler(consumerKey, consumerSecret)
        auth.set_access_token(accessToken, accessTokenSecret)
        api = tweepy.API(auth)

        # input for term to be searched and how many tweets to search
        searchTerm = input("Enter Keyword/Hashtag to search about: ")
        NoOfTerms = int(input("Number of tweets to analyze: "))

        # searching for tweets
        self.tweets = tweepy.Cursor(api.search, q=searchTerm, lang = "en").items(NoOfTerms)

        # Open/create a file to append data to
        #csvFile = open('result.csv', 'a')

        # Use csv writer
        #csvWriter = csv.writer(csvFile)


        # Creating some variables to store data
        polarity = 0
        positive = 0
        wpositive = 0
        spositive = 0
        negative = 0
        wnegative = 0
        snegative = 0
        neutral = 0


        # iterating through tweets fetched
        for tweet in self.tweets:
            #Append to temp so that we can store in csv later. Encode in UTF-8
            self.tweetText.append(self.cleanTweet(tweet.text).encode('utf-8'))
            analysis = TextBlob(tweet.text)
            #print(analysis.sentiment)  # print tweet's polarity
            polarity += analysis.sentiment.polarity  # adding up polarities to find the average

            if (analysis.sentiment.polarity == 0):  # adding how people are reacting to find average sentiment
                neutral += 1
            elif (analysis.sentiment.polarity > 0 and analysis.sentiment.polarity <= 0.3):
                wpositive += 1
            elif (analysis.sentiment.polarity > 0.3 and analysis.sentiment.polarity <= 0.6):
                positive += 1
            elif (analysis.sentiment.polarity > 0.6 and analysis.sentiment.polarity <= 1):
                spositive += 1
            elif (analysis.sentiment.polarity > -0.3 and analysis.sentiment.polarity <= 0):
                wnegative += 1
            elif (analysis.sentiment.polarity > -0.6 and analysis.sentiment.polarity <= -0.3):
                negative += 1
            elif (analysis.sentiment.polarity > -1 and analysis.sentiment.polarity <= -0.6):
                snegative += 1


        # Write to csv and close csv file
        #csvWriter.writerow(self.tweetText)
        #csvFile.close()

        # finding average of how people are reacting
        positive = self.percentage(positive, NoOfTerms)
        wpositive = self.percentage(wpositive, NoOfTerms)
        spositive = self.percentage(spositive, NoOfTerms)
        negative = self.percentage(negative, NoOfTerms)
        wnegative = self.percentage(wnegative, NoOfTerms)
        snegative = self.percentage(snegative, NoOfTerms)
        neutral = self.percentage(neutral, NoOfTerms)

        # finding average reaction
        polarity = polarity / NoOfTerms

        # printing out data
        print("How people are reacting on " + searchTerm + " by analyzing " + str(NoOfTerms) + " tweets.")
        print()
        print("General Report: ")

        if (polarity == 0):
            print("Neutral")
        elif (polarity > 0 and polarity <= 0.3):
            print("Weakly Positive")
        elif (polarity > 0.3 and polarity <= 0.6):
            print("Positive")
        elif (polarity > 0.6 and polarity <= 1):
            print("Strongly Positive")
        elif (polarity > -0.3 and polarity <= 0):
            print("Weakly Negative")
        elif (polarity > -0.6 and polarity <= -0.3):
            print("Negative")
        elif (polarity > -1 and polarity <= -0.6):
            print("Strongly Negative")

        print()
        print("Detailed Report: ")
        print(str(spositive) + "% people have a strongly positive reaction")
        print(str(positive) + "% people have positive reaction")
        print(str(wpositive) + "% people have a weakly positive reaction")
        print(str(neutral) + "% people have a neutral reaction")
        print(str(wnegative) + "% people have a weakly negative reaction")
        print(str(negative) + "% people have a negative reaction")
        print(str(snegative) + "% people have a strongly negative reaction")
        

        self.plotPieChart(spositive, positive, wpositive, neutral, wnegative, negative, snegative, searchTerm, NoOfTerms)


    def cleanTweet(self, tweet):
        # Remove Links, Special Characters etc from tweet
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) | (\w +:\ / \ / \S +)", " ", tweet).split())

    # function to calculate percentage
    def percentage(self, part, whole):
        temp = 100 * float(part) / float(whole)
        return format(temp, '.2f')

    def plotPieChart(self, spositive, positive, wpositive, neutral, wnegative, negative, snegative, searchTerm, noOfSearchTerms):
        labels = ['Strongly Positive [' + str(spositive) + '%]','Positive [' + str(positive) + '%]', 'Weakly Positive [' + str(wpositive) + '%]', 'Neutral [' + str(neutral) + '%]',
                  'Weakly Negative [' + str(wnegative) + '%]','Negative [' + str(negative) + '%]', 'Strongly Negative [' + str(snegative) + '%]']
        sizes = [spositive, positive, wpositive, neutral, wnegative, negative, snegative]
        colors = ['darkgreen','yellowgreen','lightgreen','gold', 'lightsalmon','red','darkred']
        patches, texts = plt.pie(sizes, colors=colors, startangle=90)
        plt.legend(patches, labels, loc="best")
        plt.title('How people are reacting on ' + searchTerm + ' by analyzing ' + str(noOfSearchTerms) + ' Tweets.')
        plt.axis('equal')
        plt.tight_layout()
        plt.show()



if __name__== "__main__":
    sa = SentimentAnalysis()
    sa.DownloadData()